# 03_카드승인매출정보

In [0]:
from pyspark.sql.functions import when, col, to_date
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import re

# 한글 글꼴 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우에서 일반적으로 지원되는 한글 글꼴 (Mac의 경우 'AppleGothic' 등 사용)
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호가 깨지는 문제 해결


---

In [0]:
df = spark.read.option("multiLine", "true").csv("/mnt/raw-data/03.카드 승인매출정보/*.csv",header = True, encoding='UTF-8')
all_columns = df.columns
len(all_columns)

In [0]:
display(df)

Databricks data profile. Run in Databricks to view.

In [0]:
df.printSchema()

# 컬럼 그룹화 (대분류)
---

In [0]:
all_columns = df.columns
print(f'전체 컬럼 :{len(all_columns)}\n')

key_columns = ["기준년월", "발급회원번호"]


# 날짜 관련 컬럼 (컬럼명에 '일자'가 포함된 컬럼)
date_columns = [col for col in all_columns if "일자" in col]
print(f'날짜 관련 컬럼 :{len(date_columns)}')
date_columns += key_columns
print(f'날짜 관련 컬럼 :{len(date_columns)}\n')

# 금액 관련 컬럼 (컬럼명에 '금액'이 포함된 컬럼)
amount_columns = [col for col in all_columns if "금액" in col]
print(f'금액 관련 컬럼 :{len(amount_columns)}')
amount_columns += key_columns
print(f'금액 관련 컬럼 :{len(amount_columns)}\n')

# 건수 관련 컬럼 (컬럼명에 '건수'가 포함된 컬럼)
count_columns = [col for col in all_columns if "건수" in col]
print(f'건수 관련 컬럼 :{len(count_columns)}')
count_columns += key_columns
print(f'건수 관련 컬럼 :{len(count_columns)}\n')

# 이용후경과월 관련 컬럼
use_after_month_columns = [col for col in all_columns if "이용후경과월" in col]
print(f'경과 관련 컬럼 :{len(use_after_month_columns)}')
use_after_month_columns += key_columns
print(f'경과 관련 컬럼 :{len(use_after_month_columns)}\n')

# 이용개월수 관련 컬럼
use_month_columns = [col for col in all_columns if "이용개월수" in col]
print(f'이용개월 관련 컬럼 :{len(use_month_columns)}')
use_month_columns += key_columns
print(f'이용개월 관련 컬럼 :{len(use_month_columns)}\n')


In [0]:
# 기타 컬럼 (위 리스트들에 포함되지 않은 나머지 컬럼들)
other_columns = [col for col in all_columns if col not in (date_columns + amount_columns + count_columns + use_after_month_columns + use_month_columns)]
print(f'기타 컬럼 :{len(other_columns)}')
other_columns += key_columns
print(f'기타 컬럼 :{len(other_columns)}\n')

In [0]:
# 빠진 컬럼 없는지 확인 # 440이어야 함
len(date_columns + amount_columns + count_columns + use_after_month_columns + use_month_columns+ other_columns)

## ✅ 날짜 관련 데이터
---
10101은 결측치로 대체해도 좋을 듯?

 ㄴ 찾아보니 일반적으로 쓰는 더미데이터 중 하나라네요 (1901년 1월 1일) 결측치로 처리하면 될 것 같습니다.

df[date_columns] = df[date_columns].replace(10101, np.nan)

In [0]:
# 1. 날짜 관련 테이블 생성
date_df = df.select(*date_columns)

In [0]:
# 1. 키 컬럼들
key_columns = ["기준년월", "발급회원번호"]

# 2. 전체 date_columns에서 키 컬럼 제외
date_only_columns = [col for col in date_columns if col not in key_columns]

# 3. 최종 컬럼 순서: 키 컬럼 먼저, 그 다음 날짜 컬럼들
ordered_columns = key_columns + date_only_columns

# 4. 원하는 순서로 DataFrame 생성
final_df = df.select(*ordered_columns)

print("컬럼 순서:")
for i, col_name in enumerate(ordered_columns):
    print(f"{i+1}. {col_name}")

display(final_df)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, to_date

# 원본 DataFrame 로드 (ps_df로 가정)
# ps_df = spark.read.table("your_table_name")  # 실제 테이블 이름으로 교체

# 1. 변경하지 않을 컬럼 리스트
exclude_columns = ["기준년월", "발급회원번호"]

# 2. DateType으로 변경할 컬럼 리스트
columns_to_convert_to_date = [
    "최종이용일자_기본",
    "최종이용일자_신판",
    "최종이용일자_CA",
    "최종이용일자_카드론",
    "최종이용일자_체크",
    "최종이용일자_일시불",
    "최종이용일자_할부",
    "최종카드론_대출일자"
]

print("=== 날짜 컬럼 타입 변환 시작 ===")
print(f"변환할 컬럼 수: {len(columns_to_convert_to_date)}개")
print(f"변환 대상 컬럼: {columns_to_convert_to_date}")

# 변환된 DataFrame을 저장할 변수
df_converted = final_df

# 각 날짜 컬럼을 순회하며 '10101'를 '19000101'로 변경한 후 DateType으로 변환
for col_name in columns_to_convert_to_date:
    df_converted = df_converted.withColumn(
        col_name,
        when(col(col_name).isNull() | (col(col_name) == "10101"), "19000101")  # '10101'를 '19000101'로 변경
        .otherwise(col(col_name))  # 다른 값은 그대로
        .cast("string")  # 문자열로 캐스팅 후 DateType 변환
    )
    
    df_converted = df_converted.withColumn(
        col_name,
        to_date(col(col_name), "yyyyMMdd")  # '19000101' 또는 원래 날짜를 DateType으로 변환
    )
    
    print(f"'{col_name}' 컬럼 변환 완료.")

print("\n=== 변환 결과 확인 ===")
print("변환 후 스키마:")
df_converted.printSchema()

print("\n변환된 날짜 컬럼 데이터 예시:")
df_converted.select(exclude_columns + columns_to_convert_to_date).show(5, truncate=False)

print("\n변환 후 NULL 값 개수 (날짜 컬럼):")
df_converted.select([F.sum(F.col(c).isNull().cast("int")).alias(f"{c}_null_count") 
                     for c in columns_to_convert_to_date]).show()

In [0]:
display(df_converted)

In [0]:
spark.sql("""
          DROP table IF EXISTS database_pjt.3_date
          """)

In [0]:
### 데이터 베이스 사용 설정
spark.sql("USE database_pjt")
print("현재 데이터베이스를 'database_pjt'로 설정")

df_converted.write.mode("overwrite").saveAsTable("3_date")

In [0]:
# from pyspark.sql.functions import when, col, to_date

# for c in date_df.columns:
#     date_df = date_df.withColumn(c, when(col(c) == "10101", None).otherwise(col(c)))
#     date_df = date_df.withColumn(c, to_date(col(c), "yyyyMMdd"))

In [0]:
# # database_3_cache 데이터베이스에 테이블 저장
# date_df.write.mode("overwrite").saveAsTable("database_03_cache.date_df")
# print("날짜 관련 테이블 생성 완료")

## ✅ 금액 관련 컬럼
---
date_columns, **amount_columns**, count_columns, use_after_month_column, use_month_columns, other_columns

In [0]:
# # 금액 관련 컬럼 (컬럼명에 '금액'이 포함된 컬럼)
# amount_columns = [col for col in all_columns if "금액" in col]
print(f'금액 관련 컬럼 :{len(amount_columns)}\n')

In [0]:
# 2. 금액 관련 테이블 생성

amount_df = df.select(*amount_columns)

### 2-1. 데이터형 변환

In [0]:
from pyspark.sql.functions import when, col, to_date

for c in amount_df.columns[:-2]:
    amount_df = amount_df.withColumn(c, col(c).cast("float"))

In [0]:
display(amount_df)
# 데이터 프로필 로드하는데 시간이 오래걸림 ()

Databricks data profile. Run in Databricks to view.

#### 💡 데이터 추출 (amount_df_)

In [0]:
# database_3_cache 데이터베이스에 테이블 저장
amount_df.write.mode("overwrite").saveAsTable("database_03_cache.amount_df_")
print("금액(ALL) 관련 테이블 생성 완료")

### 2-2. 컬럼 세분화

#### ▶️ 데이터 불러오기 (a_df)

In [0]:
a_df = spark.read.table("database_03_cache.amount_df_")

#### 2.0 기간 포함/불포함 구분

- 전체 : 175개
- 분류 : 기간 포함된 컬럼 (110개) / 불포함된 컬럼 (65개)

In [0]:
amount_columns = a_df.columns
len(amount_columns)

In [0]:
### Step 1: 기간이 포함된 컬럼 정규표현식
period_pattern = re.compile(r'^(.*)_(B\d+M|R\d+M)$') # B나 R기간

### Step 2: prefix-period 딕셔너리 만들기
period_map = {}  # 기간별로 prefix를 저장할 딕셔너리
non_period_cols = []  # 기간이 포함되지 않은 컬럼

for col in amount_columns:
    match = period_pattern.match(col)
    # prefix(ex.'이용금액_신용')와 기간(ex.'R12M')을 따로 저장
    if match:
        prefix = match.group(1) # ex. '이용금액_신용'
        period = match.group(2) # ex. 'R12M'
        if prefix not in period_map:
            period_map[prefix] = {}
        period_map[prefix][period] = col
    else:
        non_period_cols.append(col)

print(175 - len(non_period_cols), period_map)
print()
print(len(non_period_cols), non_period_cols)    # 110 + 65(기준년월, 발급회원번호 포함)

In [0]:
# 모든 prefix에 대해 period key만 모아서 set으로 중복 제거
all_periods = set()

for periods in period_map.values():
    all_periods.update(periods.keys())

# 보기 좋게 숫자 정렬 (예: B0M, B1M, ..., R3M, R6M, R12M)
sorted_periods = sorted(
    list(all_periods),
    key=lambda x: (x[0], int(re.search(r'\d+', x).group()))
)

print("✔ 사용된 기간들:", sorted_periods)

**기간 unique 정리**
| 기간코드   | 의미                                               |
| ------ | ------------------------------------------------ |
| `B0M`  | **현재 기준 시점** (예: 분석 기준이 2025년 5월이면 → 2025년 5월)   |
| `B1M`  | **1개월 전** 기준 (예: 2025년 4월)                       |
| `B2M`  | **2개월 전** 기준 (예: 2025년 3월)                       |
| `R3M`  | **최근 3개월 평균치** (예: 2025년 3\~5월의 평균 이용금액)         |
| `R6M`  | **최근 6개월 평균치** (예: 2024년 12월\~2025년 5월의 평균 이용금액) |
| `R12M` | **최근 12개월 평균치** (예: 2024년 6월\~2025년 5월의 평균 이용금액) |


#### 2.1 기간 포함 컬럼
(prefix_period_map 에 담겨져 있음)

In [0]:
amount_period_columns = [col for col in amount_columns if col not in non_period_cols]

In [0]:
# 2. 금액 관련 (기간 포함) 테이블 생성

# 공통 키 컬럼 추가 (기준년월, 발급회원번호)
key_columns = ["기준년월", "발급회원번호"]

amount_period_columns += key_columns
amount_period_columns

In [0]:
ap_df = a_df.select(*amount_period_columns)

##### 💡 데이터 추출 (amount_period_df)

In [0]:
### 데이터 베이스 사용 설정
ap_df = ap_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

ap_df.write.mode("overwrite").saveAsTable("amount_period_df")
print("이용금액(기간 포함) 관련 테이블 생성 완료")

#### 2.2 기간 불포함 컬럼

In [0]:
# 2. 금액 관련 (기간 불포함) 테이블 생성

# non_period_cols += key_columns
anp_df = a_df.select(*non_period_cols)

In [0]:
anp_df = anp_df.drop('이용금액대')


##### 💡 데이터 추출

In [0]:
### 데이터 베이스 사용 설정
anp_df = anp_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

anp_df.write.mode("overwrite").saveAsTable("amount_nonperiod_df")
print("이용금액(기간 불포함) 관련 테이블 생성 완료")

## ✅ 건수 관련 컬럼
---
date_columns, amount_columns, **count_columns**, use_after_month_column, use_month_columns, other_columns

In [0]:
# # 건수 관련 컬럼 (컬럼명에 '건수'이 포함된 컬럼) + 기준년월&발급회원번호
print(f'건수 관련 컬럼 :{len(count_columns)}\n')

In [0]:
# 3. 건수 관련 테이블 생성

count_df = df.select(*count_columns)

### 3-1. 데이터형 변환

In [0]:
from pyspark.sql.functions import when, col, to_date

for c in count_df.columns[:-2]:
    count_df = count_df.withColumn(c, col(c).cast("float"))

In [0]:
display(count_df)

Databricks data profile. Run in Databricks to view.

#### 💡 데이터 추출 (count_df)

In [0]:
# database_3_cache 데이터베이스에 테이블 저장
count_df.write.mode("overwrite").saveAsTable("database_03_cache.count_df_")
print("건수(ALL) 관련 테이블 생성 완료")

### 3-2. 컬럼 세분화

#### ▶️ 데이터 불러오기 (a_df)

In [0]:
c_df = spark.read.table("database_03_cache.count_df_")

#### 3.0 기간 포함/불포함 구분

- 전체 : 135개
- 분류 : 기간 포함된 컬럼 (119개) / 불포함된 컬럼 (16개)

In [0]:
count_columns = c_df.columns
len(count_columns)

In [0]:
### Step 1: 기간이 포함된 건수 컬럼 정규표현식
count_period_pattern = re.compile(r'^(.*)_(B\d+M|R\d+M)$')

### Step 2: prefix-period 딕셔너리 만들기 (건수)
count_period_map = {}
count_non_period_cols = []

for column in count_columns:
    match = count_period_pattern.match(column)
    if match:
        prefix = match.group(1)
        period = match.group(2)
        if prefix not in count_period_map:
            count_period_map[prefix] = {}
        count_period_map[prefix][period] = column
    else:
        count_non_period_cols.append(column)

print(f"기간 포함 건수 컬럼: {len(count_columns) - len(count_non_period_cols)}개\n", count_period_map)
print(f"기간 미포함 건수 컬럼: {len(count_non_period_cols)}개\n", count_non_period_cols)

In [0]:
119+18

In [0]:
# 모든 prefix에 대해 period key만 모아서 set으로 중복 제거
count_all_periods = set()

for periods in count_period_map.values():
    count_all_periods.update(periods.keys())

# 보기 좋게 숫자 정렬 (예: B0M, B1M, ..., R3M, R6M, R12M)
count_sorted_periods = sorted(
    list(count_all_periods),
    key=lambda x: (x[0], int(re.search(r'\d+', x).group()))
)

print("✔ 사용된 기간들:", count_sorted_periods)

#### 3.1 기간 포함 컬럼

In [0]:
count_period_columns = [col for col in count_columns if col not in count_non_period_cols]

In [0]:
# 3. 건수 관련 (기간 포함) 테이블 생성

# 공통 키 컬럼 추가 (기준년월, 발급회원번호)
key_columns = ["기준년월", "발급회원번호"]

count_period_columns += key_columns
count_period_columns

In [0]:
# 121개 컬럼
cp_df = c_df.select(*count_period_columns)

##### 💡 데이터 추출 (amount_period_df)

In [0]:
### 데이터 베이스 사용 설정
cp_df = cp_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

cp_df.write.mode("overwrite").saveAsTable("count_period_df_")
print("이용건수(기간 포함) 관련 테이블 생성 완료")

#### 3.2 기간 불포함 컬럼

In [0]:
# 3. 건수 관련 (기간 불포함) 테이블 생성

# 18개 컬럼
cnp_df = c_df.select(*count_non_period_cols)

##### 💡 데이터 추출

In [0]:
### 데이터 베이스 사용 설정
cnp_df = cnp_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

cnp_df.write.mode("overwrite").saveAsTable("count_nonperiod_df_")
print("이용건수(기간 불포함) 관련 테이블 생성 완료")

## ✅ 경과 관련 컬럼
---
date_columns, amount_columns, count_columns, **use_after_month_column**, use_month_columns, other_columns

- 이용개월 관련 컬럼 :46
- 이용개월 관련 컬럼 :48

In [0]:
# 경과 관련 컬럼
print(f'경과 관련 컬럼 :{len(use_after_month_columns)}\n')

In [0]:
# 4. 경과 관련 테이블 생성

use_after_month_df = df.select(*use_after_month_columns)

#### 4-1. 데이터형 변환

In [0]:
from pyspark.sql.functions import when, col, to_date

for c in use_after_month_df.columns[:-2]:
    use_after_month_df = use_after_month_df.withColumn(c, col(c).cast("float"))

In [0]:
display(use_after_month_df)

Databricks data profile. Run in Databricks to view.

#### 💡 데이터 추출 (use_after_month_df)

In [0]:
# database_3_cache 데이터베이스에 테이블 저장
use_after_month_df.write.mode("overwrite").saveAsTable("database_03_cache.use_after_month_df")
print("경과 관련 테이블 생성 완료")

## ✅ 이용개월 관련 컬럼
---
date_columns, amount_columns, count_columns, use_after_month_column, **use_month_columns**, other_columns

In [0]:
# 5. 이용개월 관련 테이블 생성

use_month_df = df.select(*use_month_columns)    #46개

#### 5-1. 데이터형 변환

In [0]:
for col_name in use_month_columns:
    use_month_df = use_month_df.withColumn(c, col(c).cast("int"))

In [0]:
# display(use_month_df)

Databricks data profile. Run in Databricks to view.

In [0]:
print((use_month_df.count(), len(use_month_df.columns)))
use_month_df.printSchema()

## ✅ 기타 컬럼
---
other_columns

#### 6-1. 데이터형 변환

In [0]:
print("Other columns 개수:", len(other_columns))
print("Other columns 목록:")
for i, col_name in enumerate(other_columns):
    print(f"{i+1}. {col_name}")

In [0]:
### 기타 컬럼들을 데이터형별로 분류
other_df = df[other_columns]

# 샘플 데이터로 데이터형 확인
sample_other = other_df.limit(100).toPandas()

# 1. 문자열(카테고리) 컬럼
string_columns = [
    "발급회원번호",
    "_1순위업종", "_2순위업종", "_3순위업종",
    "_1순위쇼핑업종", "_2순위쇼핑업종", "_3순위쇼핑업종",
    "_1순위교통업종", "_2순위교통업종", "_3순위교통업종",
    "_1순위여유업종", "_2순위여유업종", "_3순위여유업종",
    "_1순위납부업종", "_2순위납부업종", "_3순위납부업종",
    "최종카드론_금융상환방식코드", "최종카드론_신청경로코드"
]

# 2. 정수형 컬럼 (개월수, 횟수, 건수 등)
integer_columns = [
    "기준년월", "이용가맹점수",
    "RP후경과월", "RP후경과월_통신", "RP후경과월_아파트", "RP후경과월_제휴사서비스직접판매",
    "RP후경과월_렌탈", "RP후경과월_가스", "RP후경과월_전기", "RP후경과월_보험",
    "RP후경과월_학습비", "RP후경과월_유선방송", "RP후경과월_건강", "RP후경과월_교통",
    "최초카드론이용경과월", "최종카드론이용경과월", "카드론이용월수_누적",
    "최종카드론_대출월수", "최종카드론_거치개월수",
    "이용횟수_선결제_R6M", "이용횟수_선결제_R3M", "이용횟수_선결제_B0M",
    "이용횟수_연체_R6M", "이용횟수_연체_R3M", "이용횟수_연체_B0M",
    "연속무실적개월수_기본_24M_카드", "연속유실적개월수_기본_24M_카드"
]

# 3. 실수형 컬럼 (금액, 이율 등)
float_columns = [
    "최종카드론_대출이율",
    "정상청구원금_B0M", "선입금원금_B0M", "정상입금원금_B0M", "연체입금원금_B0M",
    "정상청구원금_B2M", "선입금원금_B2M", "정상입금원금_B2M", "연체입금원금_B2M",
    "정상청구원금_B5M", "선입금원금_B5M", "정상입금원금_B5M", "연체입금원금_B5M"
]

print(f"문자열 컬럼: {len(string_columns)}개")
print(f"정수형 컬럼: {len(integer_columns)}개") 
print(f"실수형 컬럼: {len(float_columns)}개")
print(f"총 기타 컬럼: {len(string_columns) + len(integer_columns) + len(float_columns)}개")

In [0]:
from pyspark.sql.functions import col, when
from pyspark.sql import functions as F

# 문자열 컬럼 (이미 string 타입)
string_df = other_df.select(*string_columns)

print("문자열 컬럼 처리 완료")
display(string_df.limit(5))

# 정수형으로 변환
integer_df = other_df.select(*integer_columns)

for col_name in integer_columns:
    integer_df = integer_df.withColumn(col_name, col(col_name).cast("int"))

print("정수형 컬럼 처리 완료")
display(integer_df.limit(5))

# 실수형으로 변환
float_df = other_df.select(*float_columns)

for col_name in float_columns:
    float_df = float_df.withColumn(col_name, col(col_name).cast("float"))

print("실수형 컬럼 처리 완료")
display(float_df.limit(5))

# 모든 기타 컬럼을 하나로 통합
final_other_columns = string_columns + integer_columns + float_columns
final_other_df = other_df.select(*final_other_columns)

# 데이터형 변환 적용
for col_name in string_columns:
    final_other_df = final_other_df.withColumn(col_name, 
                                             when(col(col_name).isNull() | (col(col_name) == ""), "Unknown")
                                             .otherwise(col(col_name)))

for col_name in integer_columns:
    final_other_df = final_other_df.withColumn(col_name, col(col_name).cast("int"))

for col_name in float_columns:
    final_other_df = final_other_df.withColumn(col_name, col(col_name).cast("float"))

print(f"최종 기타 컬럼 수: {len(final_other_columns)}개")
print("데이터형 변환 완료")
display(final_other_df)

# 데이터형 확인
final_other_df.printSchema()

In [0]:
other_df = df[other_columns]

In [0]:
# display(other_df)

Databricks data profile. Run in Databricks to view.

In [0]:
print((other_df.count(), len(other_df.columns)))
other_df.printSchema()

In [0]:
# RP금액_B0M = 이번 달 (지난 0개월) 동안의 회전 결제 금액
#            = (지난달에 이월된 금액) + (이번달에 사용한 금액) = 이번 달 회전 결제 금액

---
---

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [0]:
date_columns = [
    "기준년월", "최종이용일자_기본", "최종이용일자_신판", "최종이용일자_CA",
    "최종이용일자_카드론", "최종이용일자_체크", "최종이용일자_일시불", "최종이용일자_할부" ]

print('날짜 관련 컬럼 :',len(date_columns))

In [0]:
amount_columns = [
    "이용금액_신용_B0M", "이용금액_신판_B0M", "이용금액_일시불_B0M", "이용금액_할부_B0M", 
    "이용금액_할부_유이자_B0M", "이용금액_할부_무이자_B0M", "이용금액_부분무이자_B0M", 
    "이용금액_CA_B0M", "이용금액_체크_B0M", "이용금액_카드론_B0M"
]

print('금액 관련 컬럼 :',len(amount_columns))

In [0]:
count_columns = [
    "이용건수_신용_B0M", "이용건수_신판_B0M", "이용건수_일시불_B0M", "이용건수_할부_B0M", 
    "이용건수_할부_유이자_B0M", "이용건수_할부_무이자_B0M", "이용건수_부분무이자_B0M", 
    "이용건수_CA_B0M", "이용건수_체크_B0M", "이용건수_카드론_B0M"
]

print('건수 관련 컬럼 :',len(amount_columns))

In [0]:
other_columns = [
    "발급회원번호", "RP건수_B0M", "RP금액_B0M", "RP유형건수_B0M", "RP건수_통신_B0M", 
    "RP건수_아파트_B0M", "RP건수_제휴사서비스직접판매_B0M", "RP건수_렌탈_B0M", 
    "RP건수_가스_B0M", "RP건수_전기_B0M", "RP건수_보험_B0M", "RP건수_학습비_B0M", 
    "RP건수_유선방송_B0M", "RP건수_건강_B0M", "RP건수_교통_B0M", "증감_RP건수_전월",
    "증감_RP유형건수_전월", "증감_RP건수_통신_전월", "증감_RP건수_아파트_전월"
]
print('기타 컬럼 :',len(other_columns))

In [0]:
from pyspark.sql.functions import concat, lit, when, to_date

# 'yyyy-MM' 형식의 '기준년월'에 '01'을 추가하여 'yyyy-MM-01' 형식으로 변환
# df = df.withColumn("기준년월", to_date(concat(df["기준년월"], lit("01")), "yyyy-MM-dd"))

# '10101' 값을 제외하고 나머지는 정상적인 날짜 형식으로 변환
df = df.withColumn("최종이용일자_기본", 
                   when(df["최종이용일자_기본"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_기본"], "yyyy-MM-dd")))

df = df.withColumn("최종이용일자_신판", 
                   when(df["최종이용일자_신판"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_신판"], "yyyy-MM-dd")))

df = df.withColumn("최종이용일자_CA", 
                   when(df["최종이용일자_CA"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_CA"], "yyyy-MM-dd")))

df = df.withColumn("최종이용일자_카드론", 
                   when(df["최종이용일자_카드론"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_카드론"], "yyyy-MM-dd")))

df = df.withColumn("최종이용일자_체크", 
                   when(df["최종이용일자_체크"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_체크"], "yyyy-MM-dd")))

df = df.withColumn("최종이용일자_일시불", 
                   when(df["최종이용일자_일시불"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_일시불"], "yyyy-MM-dd")))

df = df.withColumn("최종이용일자_할부", 
                   when(df["최종이용일자_할부"] == "10101", None)
                   .otherwise(to_date(df["최종이용일자_할부"], "yyyy-MM-dd")))


In [0]:
display(df)

In [0]:
from pyspark.sql.functions import to_date

# 날짜 컬럼들을 timestamp 형식으로 변환
df = df.withColumn("기준년월", to_date(df["기준년월"], "yyyy-MM"))
df = df.withColumn("최종이용일자_기본", to_date(df["최종이용일자_기본"], "yyyy-MM-dd"))
df = df.withColumn("최종이용일자_신판", to_date(df["최종이용일자_신판"], "yyyy-MM-dd"))
df = df.withColumn("최종이용일자_CA", to_date(df["최종이용일자_CA"], "yyyy-MM-dd"))
df = df.withColumn("최종이용일자_카드론", to_date(df["최종이용일자_카드론"], "yyyy-MM-dd"))
df = df.withColumn("최종이용일자_체크", to_date(df["최종이용일자_체크"], "yyyy-MM-dd"))
df = df.withColumn("최종이용일자_일시불", to_date(df["최종이용일자_일시불"], "yyyy-MM-dd"))
df = df.withColumn("최종이용일자_할부", to_date(df["최종이용일자_할부"], "yyyy-MM-dd"))


In [0]:
display(df)

In [0]:
df.printSchema()

In [0]:
df.select("기준년월").distinct().show()

In [0]:
# 신용카드와 체크카드의 이용 금액 합계
df = df.withColumn("이용금액_신용_체크_합계_B0M", 
                   F.col("이용금액_신용_B0M").cast("float") + F.col("이용금액_체크_B0M").cast("float"))

# 각 카드별 이용 건수 평균
df = df.withColumn("평균_이용건수_B0M", 
                   (F.col("이용건수_신용_B0M").cast("float") + F.col("이용건수_체크_B0M").cast("float")) / 2)

In [0]:
# 각 컬럼에 대한 고유값 수와 설명
columns_info = pd.DataFrame({
    "Column Name": df.columns,
    "Unique Values": [df.select(col).distinct().count() for col in df.columns],
    "Data Type": df.dtypes,
    "Null Values": df.isnull().sum(),

    "Example Value": [df[col].sample(1).values[0] for col in df.columns]

})
columns_info.reset_index(drop=True, inplace=True)

In [0]:
from pyspark.sql.functions import col

df = df.withColumn("총_이용금액", 
                   col("이용금액_신용_B0M").cast("double") + 
                   col("이용금액_할부_B0M").cast("double") + 
                   col("이용금액_일시불_B0M").cast("double"))

# 총 이용 금액을 기준으로 고객별로 집계
df.groupBy("발급회원번호").sum("총_이용금액").show()
